In [1]:
import pandas as pd
import Reports

In [2]:
report_list = Reports.get_all_reports()

FileNotFoundError: [Errno 2] No such file or directory: './normalization_text/f995667298e06d6ce2c81f0b30d0738445919c46.jsonl'

In [2]:
len(report_list)

10214

In [ ]:
title_dict_counter: dict[str, int] = {}
for report in report_list:
    for word in report.title.split(' '):
        if not word.isalpha():
            continue
        if word not in title_dict_counter:
            title_dict_counter[word] = 1
        else:
            title_dict_counter[word] += 1

In [ ]:
ds = pd.DataFrame(title_dict_counter.items(), columns=['word', 'count'])
ds = ds.sort_values(by='count', ascending=False)
words_to_remove = ['Report', 'and', 'Annual', 'Progress', 'Development', 'Review', 'Impact', 'Business', 'Activity', 'Summary', 'Statement', 'Data', 'Update', 'Supplement', 'on', 'Reporting']
ds_filtered = ds[~ds['word'].isin(words_to_remove)].head(20)

In [ ]:
ds_filtered['count'].sum()

In [ ]:
in_title_counter = 0
not_found_reports_list = []
is_found_word = False

for report in report_list:
    is_found_word = False
    
    for word in report.title.split(' '):
        if ds_filtered[ds['word'] == word].shape[0] > 0:
            in_title_counter += 1
            is_found_word = True
            break

    if not is_found_word:
        not_found_reports_list.append(report)


In [ ]:
len(not_found_reports_list)

In [5]:
from transformers import BertTokenizer
bert_cased_tokenizer = BertTokenizer.from_pretrained("bert-base-cased")

In [6]:
from transformers import BertConfig, BertForPreTraining
config = BertConfig()
model = BertForPreTraining(config)

In [7]:
from transformers import TextDatasetForNextSentencePrediction
dataset = TextDatasetForNextSentencePrediction(
    tokenizer=bert_cased_tokenizer,
    file_path="./preprocessed.txt",
    block_size = 256
)

C:\Users\saif8\miniconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:362: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir= "./output",
    overwrite_output_dir=True,
    num_train_epochs=2,
    per_gpu_train_batch_size= 16,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()
trainer.save_model("./model")

RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
No module named 'pandas.api'

In [9]:
import pandas.api

ModuleNotFoundError: No module named 'pandas.api'

In [2]:
import numpy as np
arr = np.array([1, 2, 3, 4, 5])

In [1]:
import pandas as pd
import Reports
report_list = Reports.get_all_reports()

In [8]:
report_list[0].pages_list[0].block_list[0].text_str

''

In [139]:
import re
paragraph_list = []

for report in report_list:
    for page in report.pages_list:
        sentence_list = []
        for block in page.block_list:
            # use text_str
            # remove all numbers
            # ignore if no full stop or comma
            # split by full stop
            filtered_text = re.sub(r'\d+', '', block.text_str)
            sentence_list.append(filtered_text.strip())
        
        paragraph = ' '.join(sentence_list)
        paragraph_length = len(paragraph.split())
        if paragraph_length < 20:
            continue
        paragraph_list.append(paragraph)

In [140]:
from math import ceil
from copy import deepcopy

MAX_SENTENCE_LENGTH = 512
indices_to_pop = []
for i, p in enumerate(deepcopy(paragraph_list)):
    paragraph = p.split()
    length = len(paragraph)
    if i % 100000 == 0:
        print(i, len(paragraph_list))
    if length <= MAX_SENTENCE_LENGTH:
        continue
    
    n_splits = ceil(length/MAX_SENTENCE_LENGTH)
    split_paragraphs = [' '.join(paragraph[s*MAX_SENTENCE_LENGTH:(s*MAX_SENTENCE_LENGTH)+MAX_SENTENCE_LENGTH]) for s in range(n_splits)]
    indices_to_pop.append(i)

    paragraph_list.extend(split_paragraphs)

# need to reverse first so indices dont get messed up in the original array
for i in indices_to_pop[::-1]:
    paragraph_list.pop(i)

len('Final length: 'paragraph_list)

0 605185
100000 647603
200000 694126
300000 739440
400000 782741
500000 827236
600000 870881


In [144]:
import pickle

with open('docs_NO_ID.pkl', 'wb') as handle:
    pickle.dump(paragraph_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [141]:
for i, p in enumerate(paragraph_list):
    if p == '':
        paragraph_list.pop(i)

' OUR COMMITMENT Materiality Annually, we assess the Corporate Social Responsibility (CSR) topics that are of importance to our stakeholders and those where we can have the greatest economic, social and environmental impacts at local, regional or global levels. We are guided in this assessment by GRI standards. We examined changing external factors, including regulations and standards, social challenges people face around the world, our evolving business model and the environmental impact of our products, services, processes and operations. Our assessment included an array of fact-ﬁnding forums, including interviews and workshops with internal stakeholders and Xerox leaders. We reﬂected on discussions with external stakeholders, as well as feedback from our employees and concluded: Managing operations responsibly across our value chain — from decreasing environmental impact and protecting customer privacy to promoting diversity and ensuring ethical behavior — remains a priority. Produc

In [ ]:
# getting terms and remove the quality value
f = open('./AutoPhrase.txt', 'r', encoding='utf-8')
lines = f.readlines()
f.close()


FileNotFoundError: [Errno 2] No such file or directory: './AutoPhrase.txt'

In [3]:
import string
printable = set(string.printable)

terms = []
for line in lines:
    splitted_line = line.split('\t')
    terms.append(splitted_line[1])

f = open('./terms.txt', 'w', encoding='utf-8')
for term in terms:
    ascii_term = bytes(term, 'ascii', errors='ignore').decode()
    filtered_term = ''.join((filter(lambda x: x in printable, ascii_term)))
    f.write(filtered_term)

In [ ]:
import string
printable = set(string.printable)

f = open('./preprocessed.txt', 'r', encoding='utf-8')
lines = f.readlines()
f.close()

f = open('./preprocessed.txt', 'w', encoding='utf-8')
for line in lines:
    ascii_line = bytes(line, 'ascii', errors='ignore').decode()
    filtered_line = ''.join((filter(lambda x: x in printable, ascii_line)))
    f.write(filtered_line)
f.close()